In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [57]:
import sys

In [59]:
%load_ext autoreload
%autoreload 2

In [62]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

In [11]:
fmin = 5.725e9
fMAX = 5.875e9
f0 = 5.8e9
B = fMAX - fmin
T = 600e-6
alpha = 2 * (fMAX - fmin) / T
c = 3e8

In [47]:
def sr( t, r ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    idx = np.where( t <= T/2  )
    y[idx] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * t[idx] * tau )
    
    # down ramp
    idx = np.where( T/2 < t )
    y[idx] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * t[idx] * tau )
    
    return np.real( y )

In [139]:
t = np.arange( 0, T, 100e-9)

Let us build the equivalent of 10 files of data

In [140]:
rampsPerFile = 1500
samplesPerRamp = 6000
nbFiles = 10
adc = np.zeros( nbFiles * rampsPerFile * samplesPerRamp )

In [154]:
V = 40
rg_corner = 480 # 430m, hMontefil 90m
az_corner = 186

for n in range( nbFiles * rampsPerFile):
    if n%1000 == 0:
        print(n)
    az = n * V * T
    Rn = ( rg_corner**2 + (az - az_corner)**2 )**0.5
    # azmimuth beam pattern effect
    wa = np.sinc( np.arccos( rg_corner / Rn ) )**2
    adc[ n * samplesPerRamp : (n+1) * samplesPerRamp ] = sr( t, Rn ) * wa

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [155]:
adc_reshaped = adc.reshape( (nbFiles * rampsPerFile, samplesPerRamp) )

In [156]:
RD = build_rd_from_data_sim( adc_reshaped, samplesPerRamp )

shape of the samples matrix = (15000, 6000)
Data are NOT shifted


In [157]:
ifft_RD = np.fft.ifft( RD, axis=1 )

In [158]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( ifft_RD[::10,:] ) ) )

In [146]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,500] ) ))
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,501] ) ))

In [147]:
V * rampsPerFile * nbFiles * T / 2

179.99999999999997

In [148]:
340*2**0.5

480.83261120685233